## AI-Generated-Image-Detector: A detector ensembled Swin-Transformer and CLIP

This is a ensembled model, which include a Fine-Tuned Swin-Transformer and a CLIP Feature Classifier. The details is given by the following:

### Requirements

The package `torch`, `torchvision`, `timm` and `clip` are required to be installed. The installation code is given by the following:

In [ ]:
!pip install torch torchvision timm

In [ ]:
!pip install git+https://github.com/openai/CLIP.git

### Setup

The path of the dataset is needed to be modified:

In [3]:
dataset_path = "./AIGC-Detection-Dataset"

### Reproduction Code: The part of Fine-Tuning Swin Transformer

In [7]:
import os

save_dir = 'Fine_Tuned_Swin_Models'
os.makedirs(save_dir, exist_ok=True) 
batch_size = 64
num_epochs = 100
model_name = "swinv2_small_window16_256"

In [9]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import timm
import torch.optim as optim
from torch.optim import lr_scheduler
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from torch.amp import autocast, GradScaler  
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts


writer = SummaryWriter(log_dir=f'runs/{save_dir}')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


train_transform = transforms.Compose([
    transforms.RandomResizedCrop(256, scale=(0.9, 1.0), ratio=(1.0, 1.0)),  
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4811, 0.4579, 0.4188],
        std=[0.2482, 0.2423, 0.2464]
    )
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4811, 0.4579, 0.4188],
        std=[0.2482, 0.2423, 0.2464]
    )
])


train_dataset = datasets.ImageFolder(
    root=f'{dataset_path}/train',
    transform=train_transform
)
val_dataset = datasets.ImageFolder(
    root=f'{dataset_path}/val',
    transform=val_transform
)


train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,  
    pin_memory=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)


model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=2  
)



for param in model.parameters():
    param.requires_grad = False

for name, param in model.named_parameters():
    if 'layers.3' in name:
        param.requires_grad = True
    elif 'head' in name:
        param.requires_grad = True
    elif name == "norm.weight":
        param.requires_grad = True
    elif name == "norm.bias":
        param.requires_grad = True

model.to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.05)


optimizer = torch.optim.AdamW(
    model.parameters(),  
    lr=1e-4,             
    weight_decay=1e-4    
)



scheduler = CosineAnnealingWarmRestarts(
    optimizer,
    T_0=len(train_loader),      
    T_mult=2,    
    eta_min=5e-5 
)


scaler = GradScaler("cuda")
best_acc = 0.0  
val_loss_record = []
val_accuracy_record = []

for epoch in range(num_epochs):
    
    model.train()
    running_loss = 0.0
    correct_preds = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        optimizer.zero_grad()

        with autocast("cuda"):
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        scheduler.step()

        
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct_preds += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_preds.double() / len(train_dataset)
    print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

    
    writer.add_scalar('Loss/Train', epoch_loss, epoch)
    writer.add_scalar('Accuracy/Train', epoch_acc, epoch)

    
    model.eval()
    val_running_loss = 0.0
    val_correct_preds = 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            
            with autocast("cuda"):
                outputs = model(images)
                loss = criterion(outputs, labels)

            
            val_running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            val_correct_preds += torch.sum(preds == labels.data)

    val_epoch_loss = val_running_loss / len(val_dataset)
    val_epoch_acc = val_correct_preds.double() / len(val_dataset)
    val_loss_record.append(val_epoch_loss)
    val_accuracy_record.append(val_epoch_acc)
    print(f"Epoch {epoch+1}/{num_epochs} - Validation Loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_acc:.4f}")

    
    writer.add_scalar('Loss/Validation', val_epoch_loss, epoch)
    writer.add_scalar('Accuracy/Validation', val_epoch_acc, epoch)

    
    epoch_save_path = os.path.join(save_dir, f'model_epoch_{epoch+1}.pth')
    torch.save(model.state_dict(), epoch_save_path)
    print(f"Model for epoch {epoch+1} saved to {epoch_save_path}")

    for i, param_group in enumerate(optimizer.param_groups):
        print(f"Epoch {epoch+1}/{num_epochs}, Param Group {i} Learning Rate: {param_group['lr']:.6f}")

writer.close()


Epoch 1/100 - Training: 100%|██████████| 704/704 [03:54<00:00,  3.00it/s]


Epoch 1/100 - Training Loss: 0.1762, Accuracy: 0.9766


Epoch 1/100 - Validation: 100%|██████████| 79/79 [00:31<00:00,  2.48it/s]


Epoch 1/100 - Validation Loss: 0.1386, Accuracy: 0.9920
Model for epoch 1 saved to Fine_Tuned_Swin_Models\model_epoch_1.pth
Epoch 1/100, Param Group 0 Learning Rate: 0.000100


Epoch 2/100 - Training: 100%|██████████| 704/704 [03:55<00:00,  2.99it/s]


...

Epoch 99/100 - Validation Loss: 0.1283, Accuracy: 0.9958
Model for epoch 99 saved to Fine_Tuned_Swin_Models\model_epoch_99.pth
Epoch 99/100, Param Group 0 Learning Rate: 0.000070


Epoch 100/100 - Training: 100%|██████████| 704/704 [03:25<00:00,  3.42it/s]


Epoch 100/100 - Training Loss: 0.1176, Accuracy: 0.9998


Epoch 100/100 - Validation: 100%|██████████| 79/79 [00:28<00:00,  2.76it/s]

Epoch 100/100 - Validation Loss: 0.1363, Accuracy: 0.9924
Model for epoch 100 saved to Fine_Tuned_Swin_Models\model_epoch_100.pth
Epoch 100/100, Param Group 0 Learning Rate: 0.000069


#### Model Selection

The model is selected based on the epoch with **largest** validation loss among epochs 50-100 with a validation accuracy greater than 0.99. This selection criterion is used because validation accuracy primarily reflects intra-domain performance, which does not necessarily correlate with cross-domain accuracy. The assumption is that a model with slightly weaker intra-domain accuracy might generalize better, potentially having better results in cross-domain scenarios.

In [92]:
import os
import shutil

# Filter epochs with validation accuracy over 0.99 starting from epoch 50
start_epoch = 50
end_epoch = num_epochs
filtered_metrics = []
for epoch in range(start_epoch - 1, end_epoch):
    acc = val_accuracy_record[epoch]
    if acc > 0.99:
        filtered_metrics.append((epoch + 1, acc, val_loss_record[epoch]))


if not filtered_metrics:
    print("No epochs with validation accuracy above 0.99 in the specified range.")
else:
    # Find the epoch with the largest validation loss among the filtered ones
    worst_epoch, worst_val_accuracy, worst_val_loss = max(filtered_metrics, key=lambda x: x[2])
    worst_model_path = os.path.join(save_dir, f'model_epoch_{worst_epoch}.pth')
    destination_path = './swin_model.pth'
    shutil.copy(worst_model_path, destination_path)

    print(f"Model from epoch {worst_epoch} with validation accuracy {worst_val_accuracy:.4f} "
          f"and validation loss {worst_val_loss:.4f} copied to {destination_path}.")


Model from epoch 76 with validation accuracy 0.9912 and validation loss 0.1399 copied to ./swin_model.pth.


### Reproduction Code: The part of Classification using CLIP's Feature

#### Data Augmentation

In [20]:
import os
from PIL import Image
import torchvision.transforms as transforms
from torchvision.transforms.functional import ten_crop

def apply_augmentations(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    pad_transform = transforms.Pad(padding=(0, 0, 336, 336), fill=0)  # Padding to make the image at least 336x336
    center_crop_transform = transforms.CenterCrop(336)  # Center crop to 336x336
    flip_transform = transforms.RandomHorizontalFlip(p=1)  # Horizontal flip 

    for class_dir in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_dir)
        
        if os.path.isdir(class_path):
            output_class_dir = os.path.join(output_dir, class_dir)
            if not os.path.exists(output_class_dir):
                os.makedirs(output_class_dir)

            for filename in os.listdir(class_path):
                if filename.endswith('.jpg'):
                    img_path = os.path.join(class_path, filename)
                    img = Image.open(img_path)

                    # Pad if the image is smaller than 336x336
                    if img.width < 336 or img.height < 336:
                        img = pad_transform(img)

                    # Apply center crop
                    cropped_img = center_crop_transform(img)
                    cropped_img.save(os.path.join(output_class_dir, f'{filename}_center_crop.jpg'))

                    # Apply horizontal flip
                    flipped_img = flip_transform(cropped_img)
                    flipped_img.save(os.path.join(output_class_dir, f'{filename}_flipped.jpg'))

                    # TenCrop the image (Four corners and centre with horizontal flip)
                    crops = ten_crop(img, size=(336, 336)) 
                    for idx, cropped_img in enumerate(crops):
                        cropped_img.save(os.path.join(output_class_dir, f'{filename}_crop{idx}.jpg'))


In [21]:
input_dir = f'{dataset_path}/train'
output_dir = f'{dataset_path}/train_augmented'

apply_augmentations(input_dir, output_dir)

#### Model Training

In [23]:
import torch
import clip
from PIL import Image
import os
import numpy as np

In [24]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14@336px", device)

In [25]:
def load_and_extract_features(folder_path, model, preprocess, device):
    features = []
    labels = []
    
    for label in ["0_real", "1_fake"]:
        folder = os.path.join(folder_path, label)
        for filename in os.listdir(folder):
            img_path = os.path.join(folder, filename)
            img = Image.open(img_path).convert('RGB')
            img_input = preprocess(img).unsqueeze(0).to(device)
            with torch.no_grad():
                feature = model.encode_image(img_input)
            
            features.append(feature.cpu().numpy().flatten())
            labels.append(int(label[0])) 
    
    return np.array(features), np.array(labels)


In [26]:
train_features, train_labels = load_and_extract_features(f'{dataset_path}/train_augmented', model, preprocess, device)

In [27]:
test_features, test_labels = load_and_extract_features(f'{dataset_path}/val', model, preprocess, device)

In [28]:
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
test_features_scaled = scaler.transform(test_features)

train_features_tensor = torch.tensor(train_features_scaled, dtype=torch.float32)
test_features_tensor = torch.tensor(test_features_scaled, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32).view(-1, 1)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32).view(-1, 1)

In [29]:
import pickle

with open('trained_scaler.pkl', 'wb') as f:
    pickle.dump((scaler), f)

In [30]:
import torch.nn as nn
import torch.optim as optim


class ComplexClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, output_dim=1, dropout_rate=0.5):
        super(ComplexClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU() 
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc2(x))
        
        return x


In [31]:
import torch
import copy
import torch.nn as nn
import torch.optim as optim
from torch.amp import autocast, GradScaler


input_dim = train_features_tensor.shape[1]  
model = ComplexClassifier(input_dim)
model.to(device)
criterion = nn.BCEWithLogitsLoss()  
optimizer = optim.SGD(model.parameters(), lr=9e-1, momentum=0.99, weight_decay=1e-4)  
scaler = GradScaler("cuda")
model_list = []

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    with autocast("cuda"):  
        outputs = model(train_features_tensor.to(device))
        loss = criterion(outputs, train_labels_tensor.to(device))

    scaler.scale(loss).backward()  
    scaler.step(optimizer)  
    scaler.update()  
    
    model_list.append(copy.deepcopy(model.state_dict()))
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [1/100], Loss: 0.7155
Epoch [2/100], Loss: 0.6741
Epoch [3/100], Loss: 0.6510
Epoch [4/100], Loss: 0.6285
Epoch [5/100], Loss: 0.5844
Epoch [6/100], Loss: 0.5544
Epoch [7/100], Loss: 0.5480
Epoch [8/100], Loss: 0.5470
Epoch [9/100], Loss: 0.5455
Epoch [10/100], Loss: 0.5429
Epoch [11/100], Loss: 0.5401
Epoch [12/100], Loss: 0.5373
Epoch [13/100], Loss: 0.5346
Epoch [14/100], Loss: 0.5323
Epoch [15/100], Loss: 0.5302
Epoch [16/100], Loss: 0.5284
Epoch [17/100], Loss: 0.5268
Epoch [18/100], Loss: 0.5256
Epoch [19/100], Loss: 0.5245
Epoch [20/100], Loss: 0.5235
Epoch [21/100], Loss: 0.5227
Epoch [22/100], Loss: 0.5220
Epoch [23/100], Loss: 0.5213
Epoch [24/100], Loss: 0.5207
Epoch [25/100], Loss: 0.5202
Epoch [26/100], Loss: 0.5197
Epoch [27/100], Loss: 0.5194
Epoch [28/100], Loss: 0.5190
Epoch [29/100], Loss: 0.5186
Epoch [30/100], Loss: 0.5184
Epoch [31/100], Loss: 0.5181
Epoch [32/100], Loss: 0.5179
Epoch [33/100], Loss: 0.5176
Epoch [34/100], Loss: 0.5174
Epoch [35/100], Loss: 0

In [38]:
best_accuracy = 0

for i in range(num_epochs):
    print(f"Epoch: {i + 1}")
    
    model.load_state_dict(model_list[i])
    model.eval()
    
    with torch.no_grad():
        test_outputs = model(test_features_tensor.to(device))
        test_preds = (test_outputs > 0.5).float()
        accuracy = accuracy_score(test_labels, test_preds.cpu())
        print(f"Test Accuracy on Validation Set: {accuracy * 100:.2f}%")
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model_index = i

if best_model_index != -1:
    model.load_state_dict(model_list[best_model_index])
    torch.save(model.state_dict(), './model.pth')
    print(f"Model from epoch {best_model_index + 1} with highest accuracy ({best_accuracy * 100:.2f}%) saved as ./model.pth.")
else:
    print("No model to save.")

Epoch: 1
Test Accuracy on Validation Set: 50.04%
Epoch: 2
Test Accuracy on Validation Set: 50.04%
Epoch: 3
Test Accuracy on Validation Set: 52.80%
Epoch: 4
Test Accuracy on Validation Set: 76.04%
Epoch: 5
Test Accuracy on Validation Set: 93.08%
Epoch: 6
Test Accuracy on Validation Set: 96.60%
Epoch: 7
Test Accuracy on Validation Set: 97.34%
Epoch: 8
Test Accuracy on Validation Set: 97.70%
Epoch: 9
Test Accuracy on Validation Set: 97.82%
Epoch: 10
Test Accuracy on Validation Set: 97.98%
Epoch: 11
Test Accuracy on Validation Set: 97.96%
Epoch: 12
Test Accuracy on Validation Set: 98.00%
Epoch: 13
Test Accuracy on Validation Set: 97.98%
Epoch: 14
Test Accuracy on Validation Set: 97.98%
Epoch: 15
Test Accuracy on Validation Set: 97.92%
Epoch: 16
Test Accuracy on Validation Set: 97.82%
Epoch: 17
Test Accuracy on Validation Set: 97.78%
Epoch: 18
Test Accuracy on Validation Set: 97.78%
Epoch: 19
Test Accuracy on Validation Set: 97.76%
Epoch: 20
Test Accuracy on Validation Set: 97.68%
Epoch: 21

#### Model Selection

The model with best accuracy on the validation set is selected. This is simply because the CLIP is untrained to adapt to the AIGC detection task. Therefore, the accuracy on the validation set is assumed to be largely representing the accuracy of the test set.

## Test Code

### Template of the Test Code
The test code template is shown below:

In [44]:
import tqdm
import numpy as np
from sklearn import metrics

def test(model, swin_model, test_dataset_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()
    
    y_true = []
    y_pred = []
    test_dataset = data_loader(test_dataset_path)

    with torch.no_grad():
        for img_feature, image, label in tqdm(test_dataset, desc="Testing Progress"):
            output = model(img_feature)
            clip_prob = output.item()

            swin_output = swin_model(image.unsqueeze(dim=0).to(device))
            swin_prob = torch.softmax(swin_output, dim=1)[0, 1].item()
            combined_pred = float((0.489 * clip_prob + 0.511 * swin_prob) > 0.5)  # Model Ensembling
            
            y_true.append(label)
            y_pred.append(combined_pred)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    Accuracy = metrics.accuracy_score(y_true, y_pred)

    return Accuracy

### A demo of "data_loader" and "model"
A demo of "data_loader" and "model" is shown below. You need to construct your own "data_loader" and "model".

In [46]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import os
from PIL import Image
import torchvision.transforms as transforms
from tqdm import tqdm
import clip
import pickle
import timm

class data_loader(Dataset):
    def __init__(self, data_dir):
        self.model, self.preprocess = clip.load("ViT-L/14@336px", "cuda")
        
        real_dir = os.path.join(data_dir, '0_real')
        fake_dir = os.path.join(data_dir, '1_fake')
        
        self.real_images = [os.path.join(real_dir, f) for f in os.listdir(real_dir)]
        self.fake_images = [os.path.join(fake_dir, f) for f in os.listdir(fake_dir)]
        self.image_paths = self.real_images + self.fake_images
        self.labels = [0] * len(self.real_images) + [1] * len(self.fake_images)

        with open('trained_scaler.pkl', 'rb') as f:
            self.scaler = pickle.load(f)

        self.swin_transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.4812, 0.4579, 0.4188],
                std=[0.2483, 0.2425, 0.2466]
            )
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]
        encoded_image = self.model.encode_image(self.preprocess(image).unsqueeze(dim=0).to(torch.device("cuda")))
        encoded_image = encoded_image.cpu().detach().numpy().reshape(1, -1)  # Reshape to 2D for scaler
        img_feature = self.scaler.transform(encoded_image)[0]
        img_feature_tensor = torch.tensor(img_feature).to(torch.device("cuda"))
        img_feature_tensor = img_feature_tensor.to(torch.float32) 
        image = self.swin_transform(image)
        return img_feature_tensor, image, label

class ComplexClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, output_dim=1):
        super(ComplexClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        self.relu = nn.ReLU() 
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

swin_model = timm.create_model(
    "swinv2_small_window16_256",
    pretrained=False,
    num_classes=2
).to(torch.device("cuda"))

swin_model.load_state_dict(torch.load('./swin_model.pth', weights_only=True))


model = ComplexClassifier(768).to(torch.device("cuda"))
model_path = './model.pth'
model.load_state_dict(torch.load(model_path, weights_only=True))

<All keys matched successfully>

### Testing Result

In [49]:
test_dataset_path = f'{dataset_path}/val'
test(model, swin_model, test_dataset_path)

Testing Progress: 100%|██████████| 5000/5000 [03:44<00:00, 22.27it/s]


0.989